In [1]:
import pandas as pd
import pdfkit
from xlsx2html import xlsx2html

In [2]:
import re
import time
from win32com import client
import win32com.client
from pywintypes import com_error

In [3]:
import openpyxl
import PIL
from PIL import Image
from openpyxl import load_workbook
from openpyxl import Workbook

In [4]:
# gmail imports
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import base64
import mimetypes
import os
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from email import encoders

In [5]:
# Gmail creds
SCOPES = ['https://mail.google.com/']

creds = None

In [6]:
date = '14-20 November 2022'

In [7]:
masterFile = r'Consigner Payments 26 Dec 2022 - 1 Jan 2033 .xlsx'

In [8]:
consignerSummary = pd.read_excel(masterFile,sheet_name='Summary Per Consigner')
print(type(consignerSummary))

emailArrLen = len(consignerSummary)
display(consignerSummary)


<class 'pandas.core.frame.DataFrame'>


,Order Number,Order Date,Consignor,Product Title,Variant Title,Barcode,Price,Discount,Selling Price,Commission Rate,Commission Amount,Payout Amount,Fulfillment Status (Line Item),Payment Gateway Names,Location,Unnamed: 15
0,#61989,12-27-2022 10:15,00rajit@gmail.com,Yeezy Boost 350 V2 Natural,Uk 10 / Preowned 8/10,I-5boml344tbed,6999.0,0.0,6999.0,0.225,1574.77,5424.23,fulfilled,PayGate PayWeb,Cape Town,NaT
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,#62072,12-28-2022 14:05,1609967@students.wits.ac.za,Nike Air Max 1 Travis Scott Cactus Jack Saturn...,Uk 9 / New,I-3z8ssml8m0aju2,12999.0,0.0,12999.0,0.175,2274.82,10724.18,fulfilled,Card Payment,Cape Town,NaT
3,#62020,12-28-2022 11:19,1609967@students.wits.ac.za,Nike Dunk Low Reverse UNC (W),Uk 5 / New,I-1hhdmfmlbo48tqv,6499.0,0.0,6499.0,0.175,1137.32,5361.68,fulfilled,cash,Johannesburg,NaT
4,#62412,12-31-2022 13:47,1609967@students.wits.ac.za,Jordan 4 Retro Midnight Navy,Uk 9 / New,I-aa3kbqmlbcc5xt8,8499.0,0.0,8499.0,0.175,1487.32,7011.68,fulfilled,Card Payment,Cape Town,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,#62265,12-30-2022 12:01,NaN,Slide pure uk 6 - store,NaN,NaN,3499.0,0.0,3499.0,0.000,0.00,3499.00,fulfilled,Card Payment,Cape Town,NaT
747,#62287,12-30-2022 13:21,NaN,Juice world vlone tee large white,NaN,NaN,3499.0,0.0,3499.0,0.000,0.00,3499.00,fulfilled,Card Payment,Cape Town,NaT
748,#62264,12-30-2022 12:01,NaN,Yeezy 700 wave runner uk 9 nothing on system,NaN,NaN,9499.0,0.0,9499.0,0.000,0.00,9499.00,fulfilled,Card Payment,Cape Town,NaT
749,#62298,12-30-2022 14:14,NaN,Essentials short stretch limo L nothing on system,NaN,NaN,3499.0,0.0,3499.0,0.000,0.00,3499.00,fulfilled,Card Payment,Cape Town,NaT


In [9]:
excel_file = masterFile
wb = load_workbook(excel_file, data_only = True)
sh = wb['Summary Per Consigner']
print(type(sh))
consignerSummary['Colour'] = ''
display(consignerSummary)
for i in range(len(consignerSummary)):
    
    consignerSummary.iloc[i,consignerSummary.columns.get_loc('Colour')] = sh['A'+str(i+2)].fill.start_color.index
#consignerSummary['Colour'] = sh['A'].fill.start_color.index

consign = consignerSummary[consignerSummary.Colour == 'FF00FF00']
consign

<class 'openpyxl.worksheet.worksheet.Worksheet'>


,Order Number,Order Date,Consignor,Product Title,Variant Title,Barcode,Price,Discount,Selling Price,Commission Rate,Commission Amount,Payout Amount,Fulfillment Status (Line Item),Payment Gateway Names,Location,Unnamed: 15,Colour
0,#61989,12-27-2022 10:15,00rajit@gmail.com,Yeezy Boost 350 V2 Natural,Uk 10 / Preowned 8/10,I-5boml344tbed,6999.0,0.0,6999.0,0.225,1574.77,5424.23,fulfilled,PayGate PayWeb,Cape Town,NaT,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,
2,#62072,12-28-2022 14:05,1609967@students.wits.ac.za,Nike Air Max 1 Travis Scott Cactus Jack Saturn...,Uk 9 / New,I-3z8ssml8m0aju2,12999.0,0.0,12999.0,0.175,2274.82,10724.18,fulfilled,Card Payment,Cape Town,NaT,
3,#62020,12-28-2022 11:19,1609967@students.wits.ac.za,Nike Dunk Low Reverse UNC (W),Uk 5 / New,I-1hhdmfmlbo48tqv,6499.0,0.0,6499.0,0.175,1137.32,5361.68,fulfilled,cash,Johannesburg,NaT,
4,#62412,12-31-2022 13:47,1609967@students.wits.ac.za,Jordan 4 Retro Midnight Navy,Uk 9 / New,I-aa3kbqmlbcc5xt8,8499.0,0.0,8499.0,0.175,1487.32,7011.68,fulfilled,Card Payment,Cape Town,NaT,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,#62265,12-30-2022 12:01,NaN,Slide pure uk 6 - store,NaN,NaN,3499.0,0.0,3499.0,0.000,0.00,3499.00,fulfilled,Card Payment,Cape Town,NaT,
747,#62287,12-30-2022 13:21,NaN,Juice world vlone tee large white,NaN,NaN,3499.0,0.0,3499.0,0.000,0.00,3499.00,fulfilled,Card Payment,Cape Town,NaT,
748,#62264,12-30-2022 12:01,NaN,Yeezy 700 wave runner uk 9 nothing on system,NaN,NaN,9499.0,0.0,9499.0,0.000,0.00,9499.00,fulfilled,Card Payment,Cape Town,NaT,
749,#62298,12-30-2022 14:14,NaN,Essentials short stretch limo L nothing on system,NaN,NaN,3499.0,0.0,3499.0,0.000,0.00,3499.00,fulfilled,Card Payment,Cape Town,NaT,


,Order Number,Order Date,Consignor,Product Title,Variant Title,Barcode,Price,Discount,Selling Price,Commission Rate,Commission Amount,Payout Amount,Fulfillment Status (Line Item),Payment Gateway Names,Location,Unnamed: 15,Colour
2,#62072,12-28-2022 14:05,1609967@students.wits.ac.za,Nike Air Max 1 Travis Scott Cactus Jack Saturn...,Uk 9 / New,I-3z8ssml8m0aju2,12999.0,0.0,12999.0,0.175,2274.82,10724.18,fulfilled,Card Payment,Cape Town,NaT,FF00FF00
3,#62020,12-28-2022 11:19,1609967@students.wits.ac.za,Nike Dunk Low Reverse UNC (W),Uk 5 / New,I-1hhdmfmlbo48tqv,6499.0,0.0,6499.0,0.175,1137.32,5361.68,fulfilled,cash,Johannesburg,NaT,FF00FF00
4,#62412,12-31-2022 13:47,1609967@students.wits.ac.za,Jordan 4 Retro Midnight Navy,Uk 9 / New,I-aa3kbqmlbcc5xt8,8499.0,0.0,8499.0,0.175,1487.32,7011.68,fulfilled,Card Payment,Cape Town,NaT,FF00FF00
8,#62101,12-28-2022 15:28,aaronzetler@gmail.com,Nike Dunk ID Low Panda,Uk 10 / New,I-10otnhsml9woyzrb,4999.0,0.0,4999.0,0.175,874.82,4124.18,fulfilled,EFT,Cape Town,NaT,FF00FF00
9,#62023,12-28-2022 11:21,aaronzetler@gmail.com,adidas Yeezy Slide Glow Green (2022) (Restock),Uk 10 / New,I-3n26xml3d1xtm0,2499.0,0.0,2499.0,0.175,437.32,2061.68,fulfilled,Card Payment,Johannesburg,NaT,FF00FF00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,#32-2278,12-31-2022 13:52,zanebk786@gmail.com,Nike Dunk Low LX Banana (W),Uk 8 / New,I-acj76eml8vf8x7a,4499.0,300.0,4199.0,0.175,734.82,3464.18,fulfilled,EFT,Durban,NaT,FF00FF00
718,#32-2278,12-31-2022 13:52,zanebk786@gmail.com,Nike Dunk Low LX Banana (W),Uk 5 / New,I-dxj3ml8vetfa1,4499.0,300.0,4199.0,0.175,734.82,3464.18,fulfilled,EFT,Durban,NaT,FF00FF00
719,#62446,01-01-2023 22:50,zanebk786@gmail.com,Nike Dunk Low SP Undefeated 5 On It Black,Uk 9 / New,I-cplvrml8x1woov,4499.0,0.0,4499.0,0.175,787.32,3711.68,unfulfilled,PayGate PayWeb,NaN,NaT,FF00FF00
721,#62107,12-28-2022 15:38,zayyaan23@gmail.com,Dunk Low Fossil Rose,Uk 7 / New,I-9adzmlbhnoi3w,3999.0,300.0,3699.0,0.175,647.32,3051.68,fulfilled,Card Payment,Cape Town,NaT,FF00FF00


In [10]:
# lets get number of unique email addys

# the floats as nan cause a problem so cast the column as string
#consign["Consignor"] = consign["Consignor"].astype("str")
dupeEmails = list(consign['Consignor'].to_numpy())

# someone keeps leaving a newline after some emails, we must remove them to get an accurate set
newEmails = []
for sub in dupeEmails:
    newEmails.append(sub.replace("\n", ""))

finalEmails = set(newEmails)
#finalEmails.remove("nan")
print(finalEmails)
print(len(finalEmails))

{'ssmoussa657@gmail.com', 'bhavishalramsammy@gmail.com', 'trakhwale@gmail.com', 'zayyaan23@gmail.com', 'riaaz07@gmail.com', 'alessandroguerra97@gmail.com', 'shehzaadali34@gmail.com', 'boikanyotk@yahoo.com', 'aburonaldo77@gmail.com', 'jacobus.odendaal@gmail.com', 'ankebauersnk@hotmail.com', 'houseofhoops.sa@hotmail.com', 'sibusisohlabangane0@gmail.com', 'mubeen.hendrick@gmail.com', 'chiezakenneth@gmail.com', 'akbarinaam09@yahoo.com', 'exclusive.mich@gmail.com', 'steffencupido@gmail.com', '1609967@students.wits.ac.za', 'yasin.chetty@gmail.com', 'sameegh.salasa@icloud.com', 'isaacchen1990@gmail.com', 'representer786@gmail.com', 'cameronk.powan@gmail.com', 'yusufmahomedy786@gmail.com', 'jasonkyleblackman@gmail.com', 'keenanchristians@gmail.com', 'imran.bobat911@gmail.com', 'keegzhunt@gmail.com', 'Mazibukolindo252@gmail.com', 'kinokkrindi@gmail.com', 'jhb@streetclass.co.za', 'jameelsujee@gmail.com', 'danidayusa@gmail.com', 'roygruer@gmail.com', 'pragasenchipreddy@gmail.com', 'Nazmayat@gmail

In [11]:
rslt_df = consign[consign['Consignor'] == 'renee81smith@gmail.com'] 
rslt_df

,Order Number,Order Date,Consignor,Product Title,Variant Title,Barcode,Price,Discount,Selling Price,Commission Rate,Commission Amount,Payout Amount,Fulfillment Status (Line Item),Payment Gateway Names,Location,Unnamed: 15,Colour
395,#61988,12-27-2022 09:07,renee81smith@gmail.com,Nike Dunk Low Swoosh Shadow Iron Grey (GS),Uk 6 / New,I-3mj490ml99ke0fe,2299.0,0.0,2299.0,0.175,402.32,1896.68,fulfilled,PayGate PayWeb,Johannesburg,NaT,FF00FF00
396,#62159,12-29-2022 11:42,renee81smith@gmail.com,Nike Dunk Low SE Animal Instinct Black (W),Uk 6 / New,XcijyPTOL,2199.0,0.0,2199.0,0.175,384.82,1814.18,fulfilled,Card Payment,Durban,NaT,FF00FF00
397,#62253,12-30-2022 11:26,renee81smith@gmail.com,Nike Dunk High ‘Panda’ (2021) (W),Uk 8 / New,I-scrcjml9jman31,3299.0,0.0,3299.0,0.175,577.32,2721.68,fulfilled,Card Payment,Johannesburg,NaT,FF00FF00
398,#62376,12-31-2022 10:13,renee81smith@gmail.com,Nike Shox BB4 Toronto Raptors,Uk 6 / New,_FjZldOLK,3599.0,0.0,3599.0,0.175,629.82,2969.18,fulfilled,Card Payment,Cape Town,NaT,FF00FF00


In [12]:
temp_file = 'Payment Advice Template Blank.xlsx' 
wb2 = load_workbook(temp_file, data_only = True)
sh2 = wb2['Sheet2']
#copy = sh2 # we will work wth the copy from now on
#print(copy)

sheets = wb2.sheetnames # ['Sheet1', 'Sheet2']

for s in sheets:

    if s != 'Sheet2':
        sheet_name = wb2[s]
        wb2.remove(sheet_name)

# your final wb with just Sheet1



In [13]:
##################################################################################
#        ___                          __  __ 
#  ,"___".   _ _____      ___ _   LJ  LJ 
#  FJ---L]  J '_  _ `,   F __` L      FJ 
# J |  [""L | |_||_| |  | |--| |  FJ J  L
# | \___] | F L LJ J J  F L__J J J  LJ  L
# J\_____/FJ__L LJ J__LJ\____,__LJ__LJ__L
#  J_____F |__L LJ J__| J____,__F|__||__|
##################################################################################


def sendMail(email,date,htmlTable):
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().labels().list(userId='me').execute()
        labels = results.get('labels', [])

        #if not labels:
            #print('No labels found.')
            #return
        #print('Labels:')
        #for label in labels:
            #print(label['name'])
        ##########################################################################    
        mime_message = MIMEMultipart()
        
       #body
        body = f'''Good day,

We hope this email finds you well.

Please find your payment advice below.

We have already effected the payment; it should reflect in your account over the next two days.

Feel free to respond to this email with any queries that you may have.

Thanks for choosing to sell with us!

Kind Regards,

Seller Support
        '''
        # embed and hyperlink picture
        html1 = """\
        <html>
        <head></head>
        <body>
        <a href="https://courtorder.co.za/">
        <div style="display:block; text-align:center">
        <a href="https://courtorder.co.za/"> <img src="https://lh3.googleusercontent.com/Z8xy-o9VmyKN33WKBNpMG5HB2aqwFFk1hfhk0yCUomNV36jPai1oy6P3-P1ee72DR6vwDn-4zmm-L6mLWuQJvzOn3quDOFne2Ou_weryATDBZf7i9en-3CBm2kY5JHJR8PToMhUN=w596-h315-p-k" width="60%"/> </a></div>
        </a>
        </body>
        </html>
        """
        #HTML
        html2 = """\
        <html>
        <head></head>
        <body>
        
        <div style="display:block; text-align:center">
        </div>
        <div style="display:block; text-align:center">
        <a href="https://lh3.googleusercontent.com/EslpMcpT3VVj9yh_yg9C1-FyGc7wUOUu7ED2g6sYaP6QGkfZ6IKF3Rww0Am5iczfSMXGpXaldOAplqdsFhiZGXYk28Ntzy2C-Rmnz9H5UjzQZcJTWcO3IUHFSnj7C8OIaoWTvqG_=w2400?source=screenshot.guru"> <img src="https://lh3.googleusercontent.com/EslpMcpT3VVj9yh_yg9C1-FyGc7wUOUu7ED2g6sYaP6QGkfZ6IKF3Rww0Am5iczfSMXGpXaldOAplqdsFhiZGXYk28Ntzy2C-Rmnz9H5UjzQZcJTWcO3IUHFSnj7C8OIaoWTvqG_=w320-h315-p-k" width="25" height="25"/> </a>
        <p>sell@courtorder.co.za</p>
        </div>
        <div style="display:block; text-align:center">
        <a href="https://lh3.googleusercontent.com/NVof8WV5qLq79yN_vWp_RLI8U4KLdACx1KyDeGZO_tzGZMhhzKMMavxM-Pt3vXsBcFcHWf7r5M6g7syB8Rb5TTvUrgWm9Gnm6IIjN5jN7ea0fXHJ0M22CY6Zza7AikMaLK4wNN3d=w2400?source=screenshot.guru"> <img src="https://lh3.googleusercontent.com/NVof8WV5qLq79yN_vWp_RLI8U4KLdACx1KyDeGZO_tzGZMhhzKMMavxM-Pt3vXsBcFcHWf7r5M6g7syB8Rb5TTvUrgWm9Gnm6IIjN5jN7ea0fXHJ0M22CY6Zza7AikMaLK4wNN3d=w600-h315-p-k" width="25" height="25"/> </a>
        <p>www.courtorder.co.za</p>
        </div>
        </body>
        </html>"""
        
        html3 = """\
        <html>
        <head></head>
        <body>
        <div style="display:block; text-align:center; height:20px">
        </div>
        </body>
        </html>
        """
        
        
        # headers
        mime_message['To'] = f'{email}'
        mime_message['From'] = 'bilalcassim0@gmail.com'
        mime_message['Subject'] = f'Court Order Payment Advice: {date}'
        
        # This example assumes the image is in the current directory
        #fp = open('https://photos.app.goo.gl/PEEaeKAxk9V2MpAWA', 'rb')
        #msgImage = MIMEImage(fp.read())
        #fp.close()
        
        # Define the image's ID as referenced above
        #msgImage.add_header('Content-ID', '<image1>')
        #mime_message.attach(msgImage)

        # text plus HTML
       
        
        mime_message.attach(MIMEText(body, 'plain'))
        mime_message.attach(MIMEText(html1, 'html'))
        mime_message.attach(MIMEText(htmlTable, 'html'))
        
        mime_message.attach(MIMEText(html3, 'html'))
       
        # attachment
       ## attachment_filename = f''
        
        # open the file in bynary
        #binary_pdf = open(attachment_filename, 'rb')
        #payload = MIMEBase('application', 'octate-stream', Name=attachment_filename)
        #payload.set_payload((binary_pdf).read())
        
        # enconding the binary into base64
        #encoders.encode_base64(payload)
        
        # add header with pdf name
        #payload.add_header('Content-Decomposition', 'attachment', filename=attachment_filename)
        #mime_message.attach(payload)
        
        
        
        # guessing the MIME type
        #type_subtype, _ = mimetypes.guess_type(attachment_filename)
        #maintype, subtype = type_subtype.split('/')

        #with open(attachment_filename, 'rb') as fp:
        #    attachment_data = fp.read()
        #mime_message.add_attachment(attachment_data, maintype, subtype)

        encoded_message = base64.urlsafe_b64encode(mime_message.as_bytes()).decode()

        create_draft_request_body = {
            'message': {
                'raw': encoded_message
           }
        }
        # pylint: disable=E1101
        draft = service.users().drafts().create(userId="me",
                                                body=create_draft_request_body)\
            .execute()
        print(F'Draft id: {draft["id"]}\nDraft message: {draft["message"]}')
    
    except HttpError as error:
        print(F'An error occurred: {error}')
        draft = None
    return draft
    


In [14]:
#percentage
def make_percent(x):
    return str(x * 100) + '%'

def make_rand(x):
    return 'R' + str("%0.2f" % (x,)) 
    

In [15]:
#lets create a new xlsx sheet for each person
# each payment advice should have cols
#Description
#Variant
#Code
#List Price
#Discount
#Selling Price
# % com
# comm
# payout
for i in finalEmails:
    
    #print(count)
    
    temp_file = 'Payment Advice Template Blank.xlsx' 

    wb2c = load_workbook(temp_file, data_only = True)
    sh2c = wb2c['Sheet2']
    print(i)
    rslt_df = consign[consign['Consignor'] == i] 
    df2=rslt_df.reindex(columns= ['Product Title', 
                                  'Variant Title', 
                                  'Barcode',
                                  'Price',
                                  'Discount',
                                  'Selling Price',
                                  'Commission Rate',
                                  'Commission Amount',
                                  'Payout Amount'])
    
    total = round(df2['Payout Amount'].sum(),2)
    print(total)
    #print(str(df2.iloc[0,0]))
    #copy = sh2
    #####################################################################################################
    #run some functions on the columns
    #####################################################################################################
    #percentage
    df2['Commission Rate'] = df2['Commission Rate'].apply(make_percent)

    #Rands sign
    df2['Price'] = df2['Price'].apply(make_rand)
    df2['Discount'] = df2['Discount'].apply(make_rand)
    df2['Selling Price'] = df2['Selling Price'].apply(make_rand)

    df2['Commission Amount'] = df2['Commission Amount'].apply(make_rand)
    df2['Payout Amount'] = df2['Payout Amount'].apply(make_rand)
        
     
        
        
        
        
        
        
        
        
        
        
        
    #####################################################################################################
    #CALCULATIONS
    #####################################################################################################
    count = 0
    df3 = pd.DataFrame([['','','','','','','','','']], columns= ['Product Title',
                                                                 'Variant Title', 
                                                                 'Barcode',
                                                                 'Price',
                                                                 'Discount',
                                                                 'Selling Price',
                                                                 'Commission Rate',
                                                                 'Commission Amount',
                                                                 'Payout Amount'])
    display(df3)
    df3['Payout Amount'].loc[df3.index[0]] = 'R' + str("%0.2f" % (total,))
    df3['Commission Amount'].loc[df3.index[0]] = 'Total'
    
    
    df4 = pd.concat([df2, df3])
    df4.rename(columns = {'Price':'List_Price', 
                          'Selling Price':'Sale_Price',
                          'Commission Rate':'Comm_Rate',
                          'Commission Amount':'Comm',
                          'Product Title':'Product',
                          'Variant Title':'Variant',
                          'Payout Amount':'Payout'}, inplace = True)
    
    
    
    htmlFrame = df4.to_html(index=False)
    #replace text-align: right;
    #border: 1px solid #ddd; padding: 8px;

    htmlFrame = htmlFrame.replace("text-align: right;","padding-top: 12px; padding-bottom: 12px; text-align: center; background-color: #000000; color: white;")
    htmlFrame = htmlFrame.replace('<table border="1" class="dataframe">','<table style="font-family: Arial, Helvetica, sans-serif; border-collapse: collapse;width: 100%;">')
    htmlFrame = htmlFrame.replace('<th>','<th style="border: 1px solid #ddd; padding: 8px;">')
    htmlFrame = htmlFrame.replace('<td>','<td style="border: 1px solid #ddd; padding: 8px; text-align: center">')
    #print(htmlFrame)
    display(df4)
    #sh2c.cell(row=13, column=1).value = str(i) 
    #count = 0
    #for y in range(len(rslt_df)):
        #print(y)
        #for x in range(9):
            #try:
            #if (x==3 or x==4 or x==5 or x==7 or x==8):
                #sh2c.cell(row=y+17, column=x+1).value = round(float(df2.iloc[y,x]),2)
            #else:
                #sh2c.cell(row=y+17, column=x+1).value = str(df2.iloc[y,x])
        #count += round(float(str(df2.iloc[y,8])),2)
    # calculate totals
    #sh2c.cell(row=len(rslt_df)+18, column=9).value = count
    #wb2c.save('excels/test_'+i+'.xlsx')

    sendMail(i,date,htmlFrame)
    
    

    
    



ssmoussa657@gmail.com
3299.18


,Product Title,Variant Title,Barcode,Price,Discount,Selling Price,Commission Rate,Commission Amount,Payout Amount
0,,,,,,,,,


,Product,Variant,Barcode,List_Price,Discount,Sale_Price,Comm_Rate,Comm,Payout
464,Yeezy Slide Pure (2021),Uk 9 / New,PIhD0DOdg,R3999.00,R0.00,R3999.00,17.5%,R699.82,R3299.18
0,,,,,,,,Total,R3299.18


Draft id: r-9194582649043154945
Draft message: {'id': '18597d7dc4a017e3', 'threadId': '18597d7dc4a017e3', 'labelIds': ['DRAFT']}
bhavishalramsammy@gmail.com
7190.93


,Product Title,Variant Title,Barcode,Price,Discount,Selling Price,Commission Rate,Commission Amount,Payout Amount
0,,,,,,,,,


...

,Product,Variant,Barcode,List_Price,Discount,Sale_Price,Comm_Rate,Comm,Payout
634,Jordan 11 Low Concord Bred,Uk 8 / New,I-3mj490ml99lwtme,R3999.00,R0.00,R3999.00,17.5%,R699.82,R3299.18
0,,,,,,,,Total,R3299.18


Draft id: r-2398762960516266115
Draft message: {'id': '18597d7e8fd285d6', 'threadId': '18597d7e8fd285d6', 'labelIds': ['DRAFT']}
zayyaan23@gmail.com
5608.41


,Product Title,Variant Title,Barcode,Price,Discount,Selling Price,Commission Rate,Commission Amount,Payout Amount
0,,,,,,,,,


,Product,Variant,Barcode,List_Price,Discount,Sale_Price,Comm_Rate,Comm,Payout
721,Dunk Low Fossil Rose,Uk 7 / New,I-9adzmlbhnoi3w,R3999.00,R300.00,R3699.00,17.5%,R647.32,R3051.68
722,Fear of God Essentials T-shirt Egg Shell,L / New,I-34wu1ml9i7dc4a,R3499.00,R200.00,R3299.00,22.5%,R742.27,R2556.73
0,,,,,,,,Total,R5608.41


KeyboardInterrupt: 